# 구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 데이터 가져오기

### **변수명 / 설명**

* num : NO / 분류번호(일련번호)
* category : code3 / 분류번호(업종)
    ```
    ①농림축산업
    ②광업
    ③제조업
    ④건설업
    ⑤도소매업
    ⑥숙박 및 서비스업
    ⑦기타 서비스업
    ⑧공공기관
    ```
* T : code4 / 분류번호 (톤급)
    ```
    ① 1톤 미만
    ② 1톤 이상 ~ 2.5톤 미만
    ③ 2.5톤 이상 ~ 5.5톤 미만
    ④ 5.5톤 이상 ~ 8.5톤 미만
    ⑤ 8.5톤 이상 ~ 15톤 미만
    ⑥ 15톤 이상
    ```
* car_type : A2_4 / 차량종류
    ```
    ① 카고형 차량 (밴형, 탑차, 윙바디 포함)
    ② 덤프형 차량
    ③ 유조차 및 탱크로리 차량 (유류, 가스, 화공약품 등)
    ④ 특수용도형 차량 (냉장/냉동, 곡물, 사료, 활어 등)
    (직접기입: )
    ⑤ 컨테이너전용 운송차량
    ⑥ 벌크화물운송 차량[BCT] (시멘트 등)
    ⑦ 기타트레일러 차량 (철강, 목재 등)
    ```
* is_freezer : A2_4_t / 차량종류 중 냉장/냉동 1
* vibration-free : A2_5 / 무진동차량
* transport_type : A7_1 / 운송형태
    ```
    ① 편도통행
    ② 왕복통행
    ③ 다수통행
    ```
* Departure (1) : A8_1_1 / 최초출발지(시도) / <보기2> 참고
* Departure (2) : A8_1_2 / 최초출발지(시군구) / <보기2> 참고
* Departure (3) : A8_1_3 / 최초출발지(읍면동)
* Destination (1) : B1_8_1 / 통행1 도착지(시도) / <보기2> 참고
* Destination (2) : B1_8_2 / 통행1 도착치(시군구) / <보기2> 참고
* Destination (3) : B1_8_3 / 통행1 도착지(읍면동)
* Destination type : B1_9 / 통행1 도착지 유형 / <보기3> 참고 (6 : 도매시장, 유통센터, 7 : 배송센터, 창고)
    ```
    1	철도역
    2	항만
    3	공항
    4	공장
    5	버스터미널
    6	도매시장, 유통센터
    7	배송센터, 창고
    8	트럭터미널
    9	복합화물터미널(IFT)
    10	내륙컨테이너기지(ICD)
    11	차고지
    12	주거시설
    13	의료시설
    14	교육시설
    15	종교시설
    16	일반업무시설
    17	공공업무시설
    18	숙박시설
    19	판매시설
    20	관람, 집회 시설
    21	관광, 휴게시설
    22	전시시설
    23	휴게소
    24	자택
    25	기타
    26	건설현장
    27	농림수축산지
    28	폐기물처리장
    ```

In [5]:
import pandas as pd
data = pd.read_csv('/gdrive/MyDrive/big_project/data/data_preprocessing.csv', encoding='cp949')

In [10]:
user2idx = {}
for i, l in enumerate(data['num'].unique()):
    user2idx[l] = i

dep2idx = {}
for i, l in enumerate(data['Departure (3)'].unique()):
    dep2idx[l] = i

In [11]:
idx2user = {i: user for user, i in user2idx.items()}
idx2dep = {i: item for item, i in dep2idx.items()}

In [15]:
# 빈도 수 컬럼 추가
freq = data['Departure (3)'].value_counts()
data['depfreq'] = data['Departure (3)'].map(freq)
data.head()

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type,useridx,depidx,depfreq
0,46640,3,3,1,0,2,3,11.0,166.0,송악읍,6.0,66.0,안영동,7.0,0,0,4
1,46669,3,3,1,0,2,2,8.0,111.0,팔탄면,8.0,111.0,동탄면,7.0,1,1,3
2,46673,3,6,1,0,2,3,9.0,119.0,동면,2.0,40.0,모라동,7.0,2,2,12
3,46676,5,3,1,0,2,3,6.0,67.0,관저동,4.0,50.0,북성동,6.0,3,3,1
4,46683,1,3,4,1,2,3,10.0,147.0,광혜원면,8.0,109.0,석정동,6.0,4,4,1


In [16]:
useridx = data['useridx'] = data['num'].apply(lambda x: user2idx[x]).values
depidx = data['depidx'] = data['Departure (3)'].apply(lambda x: dep2idx[x]).values
rating = data['depfreq'].values

In [19]:
import scipy
#
data_sparse = scipy.sparse.csr_matrix((rating, (useridx, depidx)), shape=(len(set(useridx)), len(set(depidx))))

In [17]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 30.8 MB/s eta 0:00:00


In [20]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

als_model = ALS(factors=20, regularization=0.08, iterations = 20)
als_model.fit(data_sparse.T)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0029425621032714844 seconds
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
!pip install tqdm

In [23]:
from tqdm import tqdm_notebook

idx2rec = {}
for u in tqdm_notebook(idx2user.keys()):
    item_rec = als_model.recommend(u, data_sparse, N=100)
    print(item_rec)
    # item_rec = [idx2dep[x[0]] for x in item_rec]
    # idx2rec[idx2user[u]] = item_rec

<ipython-input-23-94fcfc400152>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for u in tqdm_notebook(idx2user.keys()):


  0%|          | 0/4388 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

#-------------------------
# LOAD AND PREP THE DATA
#-------------------------

raw_data = pd.read_csv('/gdrive/MyDrive/big_project/data/data_preprocessing.csv', encoding='cp949')

# num값을 인덱스로 바꾸기
raw_data['num'] = range(4388)

# 출발지 빈도 수 컬럼 추가
freq = data['Departure (3)'].value_counts()
data['departure_num'] = data['Departure (3)'].map(freq)

# # Convert departure names into numerical IDs
data['user_id'] = data['num'].astype("category").cat.codes
data['departure_id'] = data['Departure (3)'].astype("category").cat.codes
# data['destination_id'] = data['Destination (3)'].astype("category").cat.codes

# # Create a lookup frame so we can get the artist names back in readable form later.
item_lookup = data[['departure_id','Departure (3)']].drop_duplicates()
item_lookup['departure_id'] = item_lookup.departure_id.astype(str)
# item_lookup['destination_id'] = item_lookup.destination_id.astype(str)
# data.info()

# # # Create lists of all users, artists and plays
# Create lists of all users, departures, tons
users = list(np.sort(data.user_id.unique()))
departure = list(np.sort(data.departure_id.unique()))
departure_freq = list(data.departure_num)
# # destination = list(np.sort(data.destination_id.unique()))

print(len(users), len(departure), len(departure_freq))
# # print(users)
# # print(departure)
# # print(destination)
# # # # Get the rows and columns for our new matrix
rows = data.num.astype(int)
cols = data.departure_id.astype(int)
# print(len(rows), len(cols))

# # Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sparse.csr_matrix((departure_freq, (rows, cols)), shape=(len(users),len(departure)))

4388 1985 4388


In [ ]:
print(data_sparse) # 행이 4388 열이 1985  / 사용자 별 출발지의 빈도

  (0, 1002)	4
  (1, 1795)	3
  (2, 518)	12
  (3, 173)	1
  (4, 195)	1
  (5, 1021)	3
  (6, 1368)	4
  (7, 1778)	5
  (8, 1439)	4
  (9, 739)	3
  (10, 1306)	3
  (11, 94)	1
  (12, 1387)	3
  (13, 816)	3
  (14, 408)	3
  (15, 313)	3
  (16, 553)	9
  (17, 1008)	12
  (18, 816)	3
  (19, 1954)	5
  (20, 913)	2
  (21, 1803)	7
  (22, 1167)	8
  (23, 256)	1
  (24, 631)	1
  :	:
  (4363, 1110)	30
  (4364, 1110)	30
  (4365, 381)	1
  (4366, 1110)	30
  (4367, 1860)	19
  (4368, 1860)	19
  (4369, 1753)	2
  (4370, 551)	1
  (4371, 319)	1
  (4372, 1378)	4
  (4373, 749)	3
  (4374, 1038)	5
  (4375, 1326)	1
  (4376, 1512)	6
  (4377, 1038)	5
  (4378, 624)	5
  (4379, 1615)	1
  (4380, 817)	1
  (4381, 273)	3
  (4382, 719)	3
  (4383, 719)	3
  (4384, 1227)	1
  (4385, 995)	1
  (4386, 719)	3
  (4387, 1842)	3


In [ ]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):

    """ Implementation of Alternating Least Squares with implicit data. We iteratively
    compute the user (x_u) and item (y_i) vectors using the following formulas:

    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))

    Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix

        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.

        iterations (int): How many times we alternate between fixing and
        updating our user and item vectors

        lambda_val (float): Regularization value

        features (int): How many latent features we want to compute.

    Returns:
        X (csr_matrix): user vectors of size users-by-features

        Y (csr_matrix): item vectors of size items-by-features
     """

    # Calculate the foncidence for each value in our data
    confidence = sparse_data * alpha_val

    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape

    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)

    I = sparse.eye(features)
    lI = lambda_val * I
    # Start main loop. For each iteration we first compute X and then Y
    for i in range(iterations):
        print ('iteration %d of %d' % (i+1, iterations))

        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Loop through all users
        for u in range(user_size):

            # Get the user row.
            u_row = confidence[u,:].toarray()

            # Calculate the binary preference p(u)
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)


        for i in range(item_size):

            # Get the item column and transpose it.
            i_row = confidence[:,i].T.toarray()

            # Calculate the binary preference p(i)
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

user_vecs, item_vecs = implicit_als(data_sparse, iterations=20, features=20, alpha_val=40)



iteration 1 of 20
iteration 2 of 20
iteration 3 of 20
iteration 4 of 20
iteration 5 of 20
iteration 6 of 20
iteration 7 of 20
iteration 8 of 20
iteration 9 of 20
iteration 10 of 20
iteration 11 of 20
iteration 12 of 20
iteration 13 of 20
iteration 14 of 20
iteration 15 of 20
iteration 16 of 20
iteration 17 of 20
iteration 18 of 20
iteration 19 of 20
iteration 20 of 20


In [ ]:
#------------------------------
# FIND SIMILAR ITEMS
#------------------------------

# Let's find similar artists to Jay-Z.
# Note that this ID might be different for you if you're using
# the full dataset or if you've sliced it somehow.
item_id = 1110

# Get the item row for Jay-Z
item_vec = item_vecs[item_id].T

# Calculate the similarity score between Mr Carter and other artists
# and select the top 10 most similar.
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

departures = []
departure_scores = []

# Get and print the actual artists names and scores
for idx in top_10:
    departures.append(item_lookup['Departure (3)'].loc[item_lookup.departure_id == str(idx)].iloc[0])
    departure_scores.append(scores[idx])

similar = pd.DataFrame({'departure': departures, 'score': departure_scores})

print(similar)

  departure     score
0       외도동  0.007287
1       연수동  0.006111
2       고수면  0.005779
3        고잔  0.004756
4       동양동  0.004741
5        영광  0.004467
6       판부면  0.004313
7        좌천  0.004269
8       예산리  0.003987
9       해미면  0.003715


In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

# pip install implicit
import implicit

# Load the data like we did before
raw_data = pd.read_table('/gdrive/MyDrive/big_project/data/data_preprocessing.csv')
raw_data.columns = ['num','Departure (3)']

# num값을 인덱스로 바꾸기
raw_data['num'] = range(4388)

data = raw_data

# # Drop NaN columns
# data = raw_data.dropna()
# data = data.copy()

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['departure'] = data['Departure (3)'].astype("category")
data['user_id'] = data['user'].cat.codes
data['departure_id'] = data['Departure (3)'].cat.codes

# departure freq mapping with its departure
freq = data['Departure (3)'].value_counts()
data['dep_freq'] = data['Departure (3)'].map(freq)

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user)
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['dep_freq'].astype(float), (data['departure_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['dep_freq'].astype(float), (data['user_id'], data['departure_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id =  #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(data['Departure (3)'].loc[data.departure_id == idx].iloc[0])


#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 2025

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print(recommendations)


## implicit의 ALS 모델 사용

In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 25.5 MB/s eta 0:00:00


In [ ]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

als_model = ALS(factors=3, regularization=0.01, iterations = 100)
als_model.fit(data_sparse.T)


/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/usr/local/lib/python3.10/dist-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0004794597625732422 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
als_model.user_factors

array([[ 1.4589269e-12,  1.5816204e-13,  1.6027350e-12],
       [ 4.5992012e-12,  6.5461339e-12,  7.2095745e-12],
       [ 2.7438590e-11,  1.3192346e-11,  4.4482994e-13],
       ...,
       [-3.8944997e-12, -7.4797191e-12,  3.1516106e-11],
       [ 1.2658957e-11,  2.1207435e-11,  3.1161837e-11],
       [ 1.7699701e-11,  2.5260764e-11,  5.3415623e-12]], dtype=float32)

In [ ]:
als_model.item_factors

array([[-7.5775974e-14,  1.1487706e-13, -1.8669323e-14],
       [-1.3551747e-13,  2.0091521e-13,  1.2097376e-14],
       [-3.9739404e-02,  5.1421296e-02, -8.4679238e-03],
       ...,
       [-4.4111968e-14,  9.5431096e-14, -5.3654184e-14],
       [-1.5445680e-13,  6.3943229e-14,  9.5046340e-14],
       [ 6.6377507e-14,  3.3358665e-14,  3.5645372e-14]], dtype=float32)

In [ ]:
a = np.dot(als_model.user_factors, als_model.item_factors.T)*15
a[0]

array([-1.8345658e-24, -2.1981605e-24, -9.5123735e-13, ...,
       -2.0288400e-24, -9.4340526e-25,  2.3886913e-24], dtype=float32)